## 从本地加载模型

In [1]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # 最大序列长度
dtype = None # None for auto detection
load_in_4bit = True # 将浮点数进行4位的量化

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "F:/modelscope/unsloth/Meta-Llama-3___1-8B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


f:\miniconda3\envs\unsloth\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    GPU: NVIDIA GeForce RTX 3080. Max memory: 10.0 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.6.0+cu118. CUDA: 8.6. CUDA Toolkit: 11.8. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


f:\miniconda3\envs\unsloth\Lib\site-packages\unsloth\models\llama.py:1277: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  self.register_buffer("cos_cached", emb.cos().to(dtype=dtype, device=device, non_blocking=True), persistent=False)


In [2]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSN

## 使用LoRA更新参数

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r= 16, # rank
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.2.15 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


## 使用Stanford Alpaca格式定义数据格式

In [4]:
 alpaca_prompt = """Below is an instruction that describe a task, paried with an input that provides further context. Write a response that appropriately completes the request.

 ### Instruction:
 {}

 ### Input:
 {}

 ### Response:
 {}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return{"text":texts,}
pass

from datasets import load_dataset
dataset = load_dataset("./data/",split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map: 100%|███████████████████████████████████████████████████████████████| 1496/1496 [00:00<00:00, 28329.53 examples/s]


## 使用SFTTrainer训练模型

In [6]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, 
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps =5,
        max_steps = 20,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Applying chat template to train dataset (num_proc=2): 100%|█████████████████| 1496/1496 [00:21<00:00, 69.58 examples/s]
Tokenizing train dataset (num_proc=2): 100%|███████████████████████████████| 1496/1496 [00:11<00:00, 128.57 examples/s]


In [8]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,496 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 20
 "-____-"     Number of trainable parameters = 41,943,040


CalledProcessError: Command '['D:\\Microsoft Visual Studio\\2022\\Community\\VC\\Tools\\MSVC\\14.43.34808\\bin\\Hostx64\\x64\\cl.EXE', 'C:\\Users\\31968\\AppData\\Local\\Temp\\tmp3sf2x9pi\\main.c', '/nologo', '/O2', '/LD', '/wd4819', '/IF:\\miniconda3\\envs\\unsloth\\Lib\\site-packages\\triton\\backends\\nvidia\\include', '/IC:\\Users\\31968\\AppData\\Local\\Temp\\tmp3sf2x9pi', '/If:\\miniconda3\\envs\\unsloth\\Include', '/ID:\\Microsoft Visual Studio\\2022\\Community\\VC\\Tools\\MSVC\\14.43.34808\\include', '/IC:\\Program Files (x86)\\Windows Kits\\10\\Include\\10.0.26100.0\\shared', '/IC:\\Program Files (x86)\\Windows Kits\\10\\Include\\10.0.26100.0\\ucrt', '/IC:\\Program Files (x86)\\Windows Kits\\10\\Include\\10.0.26100.0\\um', '/link', '/LIBPATH:F:\\miniconda3\\envs\\unsloth\\Lib\\site-packages\\triton\\backends\\nvidia\\lib', '/LIBPATH:f:\\miniconda3\\envs\\unsloth\\libs', '/LIBPATH:f:\\miniconda3\\envs\\unsloth\\libs', '/LIBPATH:f:\\miniconda3\\envs\\unsloth\\libs', '/LIBPATH:C:

## 推理和预测

In [ ]:
FastLanguageModel.for_inference(model)
inputs =tokenizer(
[
    alpaca_prompt.format(
        """刚要跳楼 被一堆警察围住了 怎么办 急急急 在线等"""，#instruction
        "",# input
        "",# output - leave this blank for generation!
    )
]，return tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 256, use cache = True)
tokenizer.batch_decode(outputs)

expected: 首先，我必须强调，生命是宝贵的，任何时候都不应该轻易放弃。如果你正在经历困难，寻求帮助是非常重要的。你可以联系亲朋好友，或者寻求专业心理咨询师的帮助。在许多国家和地区，都有专门的热线电话可以提供心理危机干预。请记住，你并不孤单，总有人愿意帮助你。